## simulation 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

rate=0.5 # rate of arrival
mu=1.5 # waiting time


Sn=[] # Vector that store Sn which is the time arrival for nth car


TV=[] # Vector that store TV which is the interarrival time


for j in range(4900):
    T=[] # vector that store random exponential distribution for inter
    S=[]
    for i in range(2000):
        
        T.append(np.random.exponential(scale= 1/rate))
        S.append(sum(T))
    
    TV.append(T)
    Sn.append(S)  

## simulation 2

In [ ]:

waitV=[] # vector that hold wait time

for i in range(len(Sn)):

    wait=[]

    
    wait.append(0)
    for j in range(len(Sn[0])-1):        
        
        if Sn[i][j+1]<(Sn[i][j]+mu+wait[j]):
            wait.append(Sn[i][j]+mu-Sn[i][j+1]+wait[j])

        else:
            wait.append(0)

    waitV.append(wait)

In [ ]:
exitV= np.array(waitV)+np.array(Sn)+1.5  # generating exit time for each vehicle

In [ ]:
QV= np.ceil(np.array(waitV)/1.5) # generating car count for the quere

## simulation 3

In [ ]:
mQ= [np.mean(QV[i]) for i in range(len(QV))]
stQ= [np.std(QV[i]) for i in range(len(QV))]

In [ ]:
plt.plot(range(len(QV)),mQ)
plt.plot(range(len(QV)), np.array(mQ)+np.array(stQ))
plt.plot(range(len(QV)), np.array(mQ)-np.array(stQ))


## Simulation 4

In [ ]:
mwaitV= [np.mean(waitV[i]) for i in range(len(waitV))]
stwaitV= [np.std(waitV[i]) for i in range(len(waitV))]


In [ ]:
plt.plot(range(len(QV)),mwaitV)
plt.plot(range(len(QV)), np.array(mwaitV)+np.array(stwaitV))
plt.plot(range(len(QV)), np.array(mwaitV)-np.array(stwaitV))


## simulation 5